In [1]:
import sys
sys.path.append('../src')
import tarski
from tarski.symbols import *

# Functional STRIPS

Functional STRIPS (FSTRIPS, for short) is a classical planning language that extends STRIPS with functional systems in order to provide a number of _expressive_ **and** _computational_ advantages, such as the ability of making indirect refernce to domain objects through the use of nested terms.

```Tarski``` lifts [Geffner's original restrictions]() on the language, and follows a full first-order logic approach. We will use the classical Blocks World domain as a running example.

### Problem Language

```Tarski``` supports the formulation of FSTRIPS problems $P$ defined over a many-sorted first-order logic language with equality. We note the language associated with a prolem $P$ as ${\cal L}(P)$.

As we have seen in previous chapters of this Tutorial, ```Tarski``` languages are made up of a finite set of sorts $T$, a possibly infinite number of variables $v_{1}^t$, $v_{2}^t, \ldots$ for each sort $t \in T$, a set $\Phi$ of _function symbols_ and a set $\Pi$ of _relation symbols_, assumed to include a number of symbols with fixed denotation, such as equality.

We can create the language ${\cal L}(P)$ for _Blocks World_ as follows

In [2]:
# 1. Create language to model the domain
bw = tarski.language('BlocksWorld')
bw.load_theory('arithmetic')

Binding operators to language: BlocksWorld: Tarski language with 4 sorts, 0 function symbols, 6 predicate symbols and 0 variables
Loaded theory 'arithmetic'


along with types $T_{blocksworld}$,

In [3]:
# 2. Define sorts that organise the objects considered
place = bw.sort('place')
block = bw.sort('block', [place])

domain specific function symbols $\phi_{blocks world}$

In [4]:
# 3. Define functions
loc = bw.function( 'loc', block, place )
width = bw.function('width', place, bw.Real)
looking_at = bw.function( 'looking_at', block )

and domain specific predicate symbols $\Pi_{blocksworld}$

In [5]:
# 4. Define predicates
clear = bw.predicate( 'clear', block)

## Problem Interpretations

The current version of ```Tarski``` requires that the [universe of discourse](https://en.wikipedia.org/wiki/Domain_of_discourse#Universe_of_discourse) in any FSTRIPS problem $P$ is **fixed**. That is, all valid interpretations ${\cal M}$ for ${\cal L}(P)$ have the same universe, denoted by ${\cal U}_P = \{ {\cal U}_t \}_{t \in T}$. It is further required that the universe ${\cal U}_t$ of any sort $t$ is _finite_, this is a limitation of the current and provably future versions of ```Tarski```.

In our model of _Blocks World_ interpretations ${\cal M}$ for ${\cal L}(P)$ consist of two sorts as defined above. We set ${\cal U}_{place}$ to $\{ b_1, b_2, b_3, b_4, table \}$ as follows



In [6]:
table = bw.constant('table', place)
b1, b2, b3, b4 = [ bw.constant('b_{}'.format(k), block) for k in (1,2,3,4) ]

Note that since ```block``` is a subtype of ```place```, every time a new constant of sort ```block``` is defined the domain of ```place``` changes accordingly. The universe of the second sort we defined, ${\cal U}_{blocks}$, has been set implicitly as well. We will collect all the blocks in a list for convenience.

In [7]:
the_blocks = [b1, b2, b3, b4]

As a result of the universe of discourse being fixed, every interpretation for the language ```bw``` shares sorts and associated constant symbols. On the other hand, the denotation of functions and predicates is subject to change. Interpretations (or models) ${\cal M}$ of ```Tarski``` languages are objects that allow to associate with function and predicate symbols their denotation. 

In [8]:
import tarski.model

M = tarski.model.create(bw)

For the purposes of this example, we will set the denotation of function and predicate symbols as follows

In [9]:
M.set( loc, (b1,), b2) # loc(b1) := b2
M.set( loc, (b2,), b3) # loc(b2) := b3
M.set( loc, (b3,), table) # loc(b3) := table
M.set( loc, (b4,), table) # loc(b4) := table

M.set( width, (b1,), 3.0) # width(b1) := 3.0
M.set( width, (b2,), 2.0) # width(b2) := 2.0
M.set( width, (b3,), 1.0) # width(b3) := 1.0
M.set( width, (b4,), 0.5) # width(b4) := 0.5
M.set( width, (table,), 100.0) # width(table) := 100.0

M.set( looking_at, tuple(), table) # looking_at() := table

M.add( clear, b1) # clear(b1)
M.add( clear, b4) # clear(b4)

## Action Schemas

A Functional STRIPS _action schema_ $a$ is **defined** by:
 - a _name_, an alphanumeric identifier,
 - a _signature_ $(v_1,\ldots,v_n)$, where each $v_i$ is a variable of some sort $T$,
 - a _precondition_ formula $\mathrm{pre}(a)$, an arbitrary formula over ${\cal L}(P)$,
 - a set of _conditional effects_ $\mathrm{effs}(a)$
 
Each effect $e \in \mathrm{effs}(a)$ can be either a _relational_ or _functional_ effect. Actions have _unspecified_ durations, measured as number of _time steps_ $dur(a)$. In Classical FSTRIPS problems durations of all actions are $1$, Differential FSTRIPS problems allow $dur(a)$ to be fixed to or bounded by an arbitrary constant. **Note**: This formalisation is still WIP.

### Relational Effects

Relational effects of actions $a$ specify _general_ properties states _must_ have after the action has been executed. The current version of ```Tarski``` requires relational effects to be of the form

$$
\phi \rightarrow L
$$

where $\phi$ is an arbitrary formula over ${\cal L}(P)$ and $L$ is a **literal**. Literals are _atomic formulas_ $R(\bar{t})$ or their negation, $\neg R(\bar{t})$ over the predicates $R$ of a given language. In the first case, we say that the effect is an _add effect_, in the second, a _delete effect_. 

Relational effects can be formalised as [Linear Temporal Logic (LTL)](http://www.cs.bham.ac.uk/research/projects/lics/) formulas such as

$$
\phi \rightarrow X L
$$

The above specifies the precise _meaning_ of the effect, that is: that if $\phi$ is true on the current state, then necessarily $L$, _as interpreted in the current state_, needs to be true in the _neXt_ state.


### Functional Effects

Functional effects of actions $a$ allow actions to set _directly_ the denotation of a given functional symbol $f(\bar{t})$ in states resulting from $a$ being done. ```Tarski``` current implementation of Functional STRIPS requires these effects to have the form

$$
\phi \rightarrow f(\bar{t}) := w
$$

where $w$ is an **arbitrary** term. $f(\bar{t})$ is made up of a function symbol $f$, of whatever arity, over a tuple of terms $\bar{t}$ of appropiate size and type. The types of the term $w$ and symbol $f$ must coincide.

The LTL specification for functional effects is as follows

$$
\phi \rightarrow X( f(\bar{t}) = w)
$$

with the same semantics, and _proviso_ with respect to interpretation of terms, as above.

### Constructing action schemas with ```Tarski```



In [10]:
import tarski.fstrips as fs
import tarski.syntax.temporal.ltl as ltl

A = []

In [11]:
dest = bw.variable('dest', block)

In [12]:
move_schema = fs.Action(bw, name='move', \
                     parameters=[dest], \
                     precondition=land(clear(looking_at()), clear(dest), width(looking_at()) < width(dest)),
                     effects = [
                                    ltl.X(loc(looking_at()) == dest)
                               ])

In [13]:
A.append(move_schema)

In [14]:
look_at_schema = fs.Action( bw, name='look_at',\
                           parameters=[dest],\
                           precondition=top,\
                           effects = [ ltl.X( looking_at() == dest)]
)

In [15]:
A.append(look_at_schema)

In [16]:
for k, a in enumerate(A) : print('{}. {}'.format(k,a))

0. action move:
pre=((clear(looking_at()) and clear(dest/block) and <(width(looking_at()),width(dest/block))))
eff=(X (=(loc(looking_at()),dest/block)))
1. action look_at:
pre=(T)
eff=(X (=(looking_at(),dest/block)))


### Constraints

The truth values of the predicate $clear(p)$ for any place $p$ need to be consistent with the following formula

$$
\forall x/place,\, \neg clear(x) \leftrightarrow x \neq table \land \exists y/block,\, loc(y)=x
$$

In [17]:
x = bw.variable('x', block)
y = bw.variable('y', block)
clear_constraint = forall( x, equiv( neg(clear(x)), land( x != table, exists(y, loc(y) == x ) )) )
print(clear_constraint)

forall x/block : (((not (not (clear(x/block))) or (!=(x/block,table) and exists y/block : (=(loc(y/block),x/block)))) and (not ((!=(x/block,table) and exists y/block : (=(loc(y/block),x/block)))) or not (clear(x/block)))))


The constraint above is binding the truth value of the predicate $clear(x)$ to two conditions:

 - $x$ is not the $table$ object, or in  other words, $clear(table)$ is _always_ true
 - there is no block $y$ on top of $x$, e.g. $loc(y) = x$

#### Discussion

Reaching this point I think it now becomes clear that syntactically, it is not really necessary to introduce an ```Axiom``` entity, yet I think using the full blown F.O. notation is a bit cumbersome. For instance, the outer universal quantification isn't strictly necessary and it would be compiled away.

Another important note is that the truth value of $clear(x)$ on a given state is the byproduct of verifying that a given state $s$ complies with the constraint i.e. the truth value of the predicate is part of the _model_ that satisfies the constraints.

### Goals

$$
loc(b_1) = loc(b_2) \land loc(b_3) = loc(b_4) \land loc(b_4) = table
$$

In [18]:
G = land( loc(b1) == loc(b2), land( loc(b3) == loc(b4), loc(b4) == table))

In [19]:
print(G)

(=(loc(b_1),loc(b_2)) and (=(loc(b_3),loc(b_4)) and =(loc(b_4),table)))


#### Discussion

This is pretty straightforward, I think.

**Next**: 

 a. [FSTRIPS with Differential Constraints](006_differential_constraints.ipynb)
 
 b. [Verification of LTL specifications and FSTRIPS](007_ltl_model_checking.ipynb)
 
 c. [FSTRIPS with Reactions or Events](008_reactions_and_events.ipynb)